# Prediction Tendency Task - Translation into Python

### Author: Merle Schuckart
### Contact: merle.schuckart@uni-luebeck.de
### Version: 9th of May, 2023
-----

#### What is this notebook about?
I would like to include Juliane Schubert's prediction tendency task in my EEG study EXNAT-2, and she was so kind to give me access to her experiment repo on Gitlab.

In her task, she "presented tone sequences of varying entropy level, to independently quantify auditory prediction tendency (as the tendency to anticipate low-level acoustic features) for each individual." (Schubert et al., 2023 --> https://doi.org/10.1093/cercor/bhac528)

Now the "problem" is that the original prediction tendency task is completely built in Matlab, whereas I wanted to have 1 single experiment file completely written in Python and not different sub-experiments in different programming languages. 

So I have to translate more or less everything into Python. I use this notebook to collect and test all the snippets before I add them to the experiment.

Juliane Schubert gave me access to her prediction tendency task experiment folder on Gitlab, but it's all in Matlab, which is not ideal because I want to use Psychopy. So I have to translate it all to Python, which is what I try to do in this notebook. :-)

----

In [14]:
# import packages
import numpy as np
from scipy.optimize import fmin_slsqp # for minimizing constrained nonlinear multivariable functions - I think that's the Python alternative for Matlab's fmincon function
import matplotlib.pyplot as plt # for testing

In [16]:
''' Helper functions '''


# Function: Find minimum of constrained nonlinear multivariable function
#def 


# Entropy Error Function:
def entropy_error(x, entropy):
  p = np.reshape(x, (n_features, n_features))  # Reshape the input vector to matrix form
  return np.sum(p * np.log(p)) - entropy  # Compute the entropy error using the reshaped matrix form of the input vector


def apply_cos_ramp(signal, fade_duration, sampling_rate):
    """Apply a cosine ramp to start & end of an audio wave.
    
    Args:
        signal (array): The audio wave to apply the cosine ramp to.
        fade_duration (float): The duration of the fade in seconds.
        sampling_rate (int): The sampling rate of the audio wave.
        
    Returns:
        array: The audio wave with the cosine ramp applied.
    """
    fade_samples = int(fade_duration * sampling_rate)
    ramp = np.cos(np.linspace(0, np.pi/2, fade_samples))
    signal[:fade_samples] *= ramp
    signal[-fade_samples:] *= ramp[::-1]
    
    return signal




In [1]:
''' Python Code for "functions/create_markov_matrix_from_entropy.m" '''

''' What does this function do? '''
# --> create a markov transition matrix, i.e. a matrix of
#     n x n values (so basically a square matrix) 
#     that reflect a certain target entropy distribution.
#     Each value is somewhere between 0 and 1, with the values in each row adding up to 1.

#     At the beginning of the function, a random Markov matrix is created. 
#     After this, a minimisation algorithm (fmincon from the MATLAB optimization toolbox) 
#     is used to modify the values in the matrix so the target entropy distribution is reached. 

#     Function Arguments: 
#            n_features = number of states or features to be modelled 
#            entropy = target entropy distribution
#            zero_elements = number of zero elements (default: 0)
#            low_bound = lowest possible value for the matrix values
#            optim_options = optimization options

#     Function Output: 
#            A Markov transition matrix that should match the target entropy distribution.




def create_markov_matrix_from_entropy(n_features, entropy, zero_elements = 0, low_bound = 0, optim_options = None):
        
    # Use default optimizer options if they are not set in the function call:
    if optim_options == None:
      # set options for optimizer
      optim_options = {'maxiter': 30000} # max. number of function evaluations allowed is 30000

    # Create a square matrix of shape n x n with 
    # random values between 0 and 1
    startmat = np.random.rand(n_features, n_features)





    # Set the lower and upper bounds of the optimization variables
    lower_bound = np.zeros((n_features, n_features)) + low_bound
    np.fill_diagonal(lower_bound, 1 / n_features)  # Set the diagonal elements to 1/n_features
    upper_bound = np.ones((n_features, n_features))
    np.fill_diagonal(upper_bound, 1 / n_features)  # Set the diagonal elements to 1/n_features
    


    # Set some elements of the upper and lower 
    # bounds to zero to force some transitions to be zero
   
    # loop number of zero elements:
    for i in range(zero_elements):
        # loop columns by looping number of features
        for cur_col in range(n_features):
            # get row index:
            idx = cur_col + i
            # if index is >= the number of columns...
            if idx >= n_features: # careful, this is > in the original function as Matlab starts counting from 1, not from 0 as in Python
                # set new row index by subtracting total number of columns from current index (this should set idx as 0 I think)
                idx = idx - n_features
            # now set upper and lower bound at row idx and current column index as 0
            upper_bound[idx, cur_col] = 0
            lower_bound[idx, cur_col] = 0


    # Define the nonlinear constraints for the optimization problem
    nonlcon = lambda x: (sum(x) - 1) + (np.sum(x, axis=1) - 1), []

    # Use fmincon to minimize the entropy error function subject to the constraints
    trans_mat = fmincon(lambda x: entropy_error(x, entropy), startmat.ravel(), [], [], [], [], lower_bound.ravel(), upper_bound.ravel(), nonlcon, options=optim_options)
    
    # Reshape the output vector to matrix form
    trans_mat = np.reshape(trans_mat, (n_features, n_features))
    
    return trans_mat





In [4]:
# DONE
''' Python Code for "+init/prepare_cfg.m" '''

''' What does this function do? '''
# --> creates dict called cfg and fills it with the 
#     specified values. Matlab's struct = Python's dict
       

def prepare_cfg():
    # This function prepares paths and configures 
    # stimulus properties and trigger values

    cfg = {}
    cfg['data_path'] = 'data'

    cfg['markov'] = {}
    cfg['markov']['n_features'] = 4
    cfg['markov']['min_prob'] = 0.05
    cfg['markov']['iti'] = 1/3
    cfg['exp'] = {}
    cfg['exp']['trials_per_block'] = 1500

    cfg['add_db'] = 40

    base_freq = 440
    fourth_ratio = 4/3

    freqs = []
    for idx_freq in range(1, 5):
        freqs.append(base_freq * (fourth_ratio**(idx_freq-1)))
    
    cfg['sounds'] = {}
    cfg['sounds']['freqs'] = freqs
    cfg['sounds']['stim_length'] = 0.1
    cfg['sounds']['fade'] = 5e-3

    cfg['triggers'] = {}
    cfg['triggers']['stims'] = [16, 32, 64, 128]

    cfg['triggers']['condition'] = {}
    cfg['triggers']['condition']['ordered'] = 1
    cfg['triggers']['condition']['random'] = 2

    # note that triggers will be added for every stimulus to provide info about
    # current entropy level! (e.g. F440 during random = 17)
    return cfg


# test function: 
print(prepare_cfg())


{'data_path': 'data', 'markov': {'n_features': 4, 'min_prob': 0.05, 'iti': 0.3333333333333333}, 'exp': {'trials_per_block': 1500}, 'add_db': 40, 'sounds': {'freqs': [440.0, 586.6666666666666, 782.2222222222222, 1042.9629629629628], 'stim_length': 0.1, 'fade': 0.005}, 'triggers': {'stims': [16, 32, 64, 128], 'condition': {'ordered': 1, 'random': 2}}}


In [ ]:
''' Python Code for "+init/prepare_stims.m" '''

''' What does this function do? '''

# 1. Creates audio stimuli using the cfg configuration object and 
#    returns a dict of audio stimuli and their trigger values.

#    The audio stimuli are created by looping over the freqs in cfg["sounds"]["freqs"], 
#    building sine waves for the current freq and duration, 
#    and applying a cosine ramp to each stimulus. 
#    The trigger names are from cfg["triggers"]["stims"]. 


# 2. Generates Markov transition matrix (using the create_markov_transition_matrix 
#    function from the exp.init module) & add it to the 
#    output dict under the name "markov". 

#     Function Argument: 
#             - cfg object (output from the custom prepare_cfg() function)

#     Function Output: 
#             - dict called "stims" containing audio stimuli and 
#               their trigger values + Markov transition matrix  


def prepare_stims(cfg):
    
    stims = {"audio": [], "trigger": []}
    
    # loop frequencies in cfg["sounds"]["freqs"]
    for i in range(len(cfg["sounds"]["freqs"])):
        
        # get current frequency:
        cur_freq = cfg["sounds"]["freqs"][i]
        print("cur_freq:", cur_freq)
        # build audio stimulus by creating a sine wave with the current frequency and duration
        # careful, o_ptb is from a MATLAB package: https://o-ptb.readthedocs.io/en/latest/
        #audio_stim = o_ptb.stimuli.auditory.Sine(cur_freq, cfg["sounds"]["stim_length"])
        
        # get duration of current stimulus from dict
        cur_duration = cfg["sounds"]["stim_length"]
        print("cur_duration:", cur_duration)
        # build a time array: you need the current duration and the right sampling frequency for your device
        # 1 divided by the sampling rate = duration of a single sample in sec
        sound_sampling_freq = 1/44100 # 44.1 kHz = 44100 Hz --> I set this at random, this is just a placeholder
        t = np.arange(0, cur_duration, sound_sampling_freq)

        # generate sine wave:
        sine_wave = np.sin(2*np.pi*cur_freq*t)
        
        # plot the sine wave
        plt.plot(t, sine_wave)
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.show()

        # Apply cosine ramp to "smoothen" the edges of the sound a bit (I'm not an audio expert as you can tell)
        # We basically gradually turn up the sound, play it for a while, 
        # and then decrease the volume again so it doesn't make annoying clicky noises when it's played.
        
        # get fade duration from config
        cur_fade_duration = cfg["sounds"]["fade"]
        print("fade:", cur_fade_duration)
        # apply cosine ramp
        sine_wave = apply_cos_ramp(sine_wave, cur_fade_duration, sound_sampling_freq)
        
        # plot the modified sine wave again
        plt.plot(t, sine_wave)
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.show()

        # append audio stimulus and corresponding trigger to "stims" dict
        #stims["audio"].append(audio_stim)
        #stims["trigger"].append(cfg["triggers"]["stims"][i])
    
    # also add markov transition matrix to the output dict 
    # under the name "trans_mats"
    #stims["markov"]["trans_mats"] = exp.init.create_markov_transition_matrix(cfg)
    
    # return dict with stims and markov transition matrix:
    return stims

  
# Test it: 
print(prepare_stims(prepare_cfg()))

In [1]:
# how to play sound in psychopy:

from psychopy import sound
import numpy as np

# Set the frequency and duration of the sine wave
cur_freq = 440 # in Hz
duration = 0.1 # in seconds

# Create a time array with the specified duration
t = np.arange(0, duration, 1/sound.getSamplingRate())

# Generate the sine wave using numpy
sine_wave = np.sin(2*np.pi*cur_freq*t)

# Create a PsychoPy sound object with the generated sine wave
audio_stim = sound.Sound(sine_wave)

# Play the sound
audio_stim.play()

# Wait for the sound to finish playing
audio_stim.wait()